In [2]:
import ctypes
import numpy as np
import sounddevice as sd
import os

In [6]:
rnnoise_dll = r"D:\Git_repos\ANC-python\rnnoise.dll"

In [7]:
dll = ctypes.WinDLL(rnnoise_dll)

try:
    for name in ["rnnoise_create", "rnnoise_destroy", "rnnoise_process_frame",
                 "rnnoise_mono_process_frame", "rnnoise_stereo_process_frame"]:
        func = getattr(dll, name, None)
        print(name, "✅ found" if func else "❌ not found")
except Exception as e:
    print("Error:", e)


rnnoise_create ✅ found
rnnoise_destroy ✅ found
rnnoise_process_frame ✅ found
rnnoise_mono_process_frame ❌ not found
rnnoise_stereo_process_frame ❌ not found


In [ ]:
# Load rnnoise.dll with correct calling convention for Windows
rnnoise = ctypes.WinDLL(rnnoise_dll)

# Define C prototypes
rnnoise.rnnoise_create.restype = ctypes.c_void_p
rnnoise.rnnoise_destroy.argtypes = [ctypes.c_void_p]
rnnoise.rnnoise_process_frame.argtypes = [
    ctypes.c_void_p,
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags='C_CONTIGUOUS')
]

# Create RNNoise state
st = rnnoise.rnnoise_create()

FRAME_SIZE = 480  # 20ms at 48kHz

def callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    # Take first channel if stereo
    indata = indata[:, 0]
    # Ensure float32 contiguous
    indata = np.ascontiguousarray(indata, dtype=np.float32)
    outbuf = np.zeros(FRAME_SIZE, dtype=np.float32)

    # Process in chunks of FRAME_SIZE
    for i in range(0, len(indata), FRAME_SIZE):
        chunk = indata[i:i+FRAME_SIZE]
        if len(chunk) < FRAME_SIZE:
            chunk = np.pad(chunk, (0, FRAME_SIZE - len(chunk)))
        rnnoise.rnnoise_process_frame(st, outbuf, chunk)
        outdata[i:i+FRAME_SIZE, 0] = outbuf[:len(chunk)]

# Open real-time stream (mic → rnnoise → speakers)
with sd.Stream(channels=1, samplerate=48000, blocksize=FRAME_SIZE,
               dtype='float32', callback=callback):
    print("Noise suppression running... press Ctrl+C to stop")
    sd.sleep(60000)  # run for 60 sec

# Clean up
rnnoise.rnnoise_destroy(st)


Noise suppression running... press Ctrl+C to stop
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflo

KeyboardInterrupt: 